---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [280]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
print(pd.__version__)

0.19.2


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [281]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [282]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # create empty DataFrame
    df_towns = pd.DataFrame(columns=["State", "RegionName"])
    
    # file name
    file_name = 'university_towns.txt'
    
    # data extraction
    with open(file_name,'r') as f:
        state = ''
        region = ''
        for row in f:
            # check if contains state or region
            if '[e' in row:
                # save state
                state = row.split('[ed')[0].strip()
                continue
            region = row.strip()
            if '(' in region:
                region = row.split('(')[0].strip()
            # populate DataFrame
            df_towns = df_towns.append({'State':state,'RegionName':region},ignore_index=True,)        
    
    return df_towns

In [283]:
def get_gdp_data():
    """ Returns a clean dataframe containing the GDP data per quarter starting in 2000q1"""
    
    # file name
    file_name = 'gdplev.xls'
    
    # load quarterly data and clean data
    df_gdp = pd.read_excel(file_name,skiprows=5,
                           index_col=0,parse_cols=[4,5,6]).dropna()
    
    index_start = df_gdp.index.get_loc('2000q1')
    
    # filter data to post 2000
    df_gdp = df_gdp.ix[index_start:]
    
    return df_gdp

In [284]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # recession dollars
    dollars = 'GDP in billions of chained 2009 dollars'
    
    # load gdp data
    df_gdp = get_gdp_data()[dollars]
    
    # find recession start
    for i in range(1,len(df_gdp)-1):
        if (df_gdp.iloc[i]<df_gdp.iloc[i-1]) and (df_gdp.iloc[i+1]<df_gdp.iloc[i]):
            return df_gdp.index[i]
    
    return None

In [285]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # recession dollars
    dollars = 'GDP in billions of chained 2009 dollars'
    
    # load gdp data
    df_gdp = get_gdp_data()[dollars]
    
    # keep data following recession start
    df_gdp = df_gdp.ix[get_recession_start():]
    
    # find recession end
    for i in range(2,len(df_gdp)-1):
        if (df_gdp.iloc[i]>df_gdp.iloc[i-1]) and (df_gdp.iloc[i-1]>df_gdp.iloc[i-2]):
            return df_gdp.index[i]
    
    return None

In [286]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # recession dollars
    dollars = 'GDP in billions of chained 2009 dollars'
    
    # load gdp data
    df_gdp = get_gdp_data()[dollars]
    
    # recession starts
    start = get_recession_start()
    end = get_recession_end()
    
    # recession data
    df_gdp = df_gdp.ix[start:end]
    
    return df_gdp.idxmin()

In [287]:
def get_housing_data():
    """Return dataframe containing the housing data from Zillow."""
    
    # file name
    file_name = 'City_Zhvi_AllHomes.csv'
    
    # load housing data and clean data
    df_housing = (pd.read_csv(file_name)
                              .replace(to_replace='NaN', value=np.NaN)
                              .convert_objects(convert_numeric=True))
    
    # remove uneccessary features
    df_housing = df_housing.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'],axis=1)
    
    # convert state ID to state name
    df_housing = df_housing.replace(states)
    
    # set multi-index
    df_housing = df_housing.set_index(['State','RegionName'])
        
    # filter columns to only contain data starting from 2000
    start_col = list(df_housing.columns.values).index('2000-01')
    df_housing = df_housing.drop(df_housing.columns[:start_col], axis=1)
    
    return df_housing

In [288]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    # import data
    df_housing = get_housing_data().T
    
    # convert index to time-stamp
    df_housing.index = pd.to_datetime(df_housing.index)
    
    # resample index by quarter and aggregate using mean
    df_housing = df_housing.resample('Q').mean()
    
    # convert index to proper values
    df_housing.index = df_housing.index.map(lambda x: str(x.year)+'q'+str((x.month-1)//3+1))
    
    # transpose data frame
    df_housing = df_housing.T
    
    return df_housing

In [289]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # load data
    df_housing = convert_housing_data_to_quarters()
    
    # find start and bottom of recession
    start_idx = df_housing.columns.get_loc(get_recession_start())
    bottom_idx = df_housing.columns.get_loc(get_recession_bottom())
    
    #(price_ratio=quarter_before_recession/recession_bottom
    df_housing['Ratio'] = df_housing[df_housing.columns[start_idx - 1]].div(df_housing[df_housing.columns[bottom_idx]])
    
    # filter data to relevant feature
    df_housing = df_housing[['Ratio']]
    
    # load univercities cities
    df_univ = get_list_of_university_towns().set_index(['State','RegionName'])
    
    # Univercity cities prices
    df_univ_prices = pd.merge(left=df_housing,right=df_univ,how='inner',
                            left_index=True,right_index=True)
    
    # generate non-university cities
    non_univ = np.setdiff1d(df_housing.index,df_univ.index)
    
    # filter data to non-university cities
    df_non_univ_prices = df_housing.loc[non_univ]
    
    # drop Na
    df_univ_prices = df_univ_prices.dropna(axis=0)
    df_non_univ_prices = df_non_univ_prices.dropna(axis=0)
    
    # T-test threshold
    alpha = 0.01
    
    # perform T-test
    s, p = ttest_ind(df_univ_prices['Ratio'], df_non_univ_prices['Ratio'])
    
    # assign output variables
    different = p<alpha
    better = "university town" if s<0 else "non-university town"
    
    return (different, p, better)